## Interaktivní Phi 3 Mini 4K Instruct Chatbot s Whisper

### Úvod:
Interaktivní Phi 3 Mini 4K Instruct Chatbot je nástroj, který uživatelům umožňuje komunikovat s ukázkou Microsoft Phi 3 Mini 4K Instruct pomocí textového nebo zvukového vstupu. Chatbot lze využít pro různé úkoly, jako je překlad, informace o počasí a obecné získávání informací.


In [ ]:
#Install required Python Packages
!pip install accelerate
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install flash-attn --no-build-isolation', env={'FLASH_ATTENTION_SKIP_CUDA_BUILD': "TRUE"}, shell=True
!pip install transformers
!pip install wheel
!pip install gradio
!pip install pydub==0.25.1
!pip install edge-tts
!pip install openai-whisper==20231117
!pip install ffmpeg==1.4
# from IPython.display import clear_output
# clear_output()

In [ ]:
# Checking to see if Cuda support is available 
# Output True = Cuda
# Output False = No Cuda (installing Cuda will be required to run the model on GPU)
import os 
import torch
print(torch.cuda.is_available())


[Vytvořte svůj Huggingface Access Token](https://huggingface.co/settings/tokens)

Vytvořte nový token  
Zadejte nový název  
Vyberte oprávnění k zápisu  
Zkopírujte token a uložte jej na bezpečné místo


Následující Python kód provádí dvě hlavní úlohy: importuje modul `os` a nastavuje proměnnou prostředí.

1. Import modulu `os`:
   - Modul `os` v Pythonu poskytuje způsob, jak komunikovat s operačním systémem. Umožňuje provádět různé úkoly související s operačním systémem, jako je přístup k proměnným prostředí, práce se soubory a adresáři atd.
   - V tomto kódu je modul `os` importován pomocí příkazu `import`. Tento příkaz zpřístupňuje funkce modulu `os` pro použití v aktuálním Python skriptu.

2. Nastavení proměnné prostředí:
   - Proměnná prostředí je hodnota, ke které mohou přistupovat programy běžící na operačním systému. Je to způsob, jak ukládat konfigurační nastavení nebo jiné informace, které mohou být využity více programy.
   - V tomto kódu je nastavena nová proměnná prostředí pomocí slovníku `os.environ`. Klíčem slovníku je `'HF_TOKEN'` a hodnota je přiřazena z proměnné `HUGGINGFACE_TOKEN`.
   - Proměnná `HUGGINGFACE_TOKEN` je definována těsně nad tímto úryvkem kódu a je jí přiřazena řetězcová hodnota `"hf_**************"` pomocí syntaxe `#@param`. Tato syntaxe se často používá v Jupyter noteboocích, aby umožnila uživatelský vstup a konfiguraci parametrů přímo v rozhraní notebooku.
   - Nastavením proměnné prostředí `'HF_TOKEN'` je možné k této hodnotě přistupovat z jiných částí programu nebo z jiných programů běžících na stejném operačním systému.

Celkově tento kód importuje modul `os` a nastavuje proměnnou prostředí s názvem `'HF_TOKEN'` na hodnotu poskytnutou v proměnné `HUGGINGFACE_TOKEN`.


In [ ]:
import os
# set the Hugging Face Token from 
# add the Hugging Face Token to the environment variables
HUGGINGFACE_TOKEN = "Enter Hugging Face Key" #@param {type:"string"}
os.environ['HF_TOKEN']HUGGINGFACE_TOKEN

Tento úryvek kódu definuje funkci nazvanou clear_output, která slouží k vymazání výstupu aktuální buňky v Jupyter Notebooku nebo IPythonu. Pojďme si rozebrat kód a pochopit jeho funkčnost:

Funkce clear_output přijímá jeden parametr nazvaný wait, což je logická hodnota (boolean). Ve výchozím nastavení je wait nastaven na hodnotu False. Tento parametr určuje, zda by funkce měla počkat, dokud nebude k dispozici nový výstup, který nahradí stávající výstup, před jeho vymazáním.

Samotná funkce slouží k vymazání výstupu aktuální buňky. V Jupyter Notebooku nebo IPythonu, když buňka generuje výstup, například vytištěný text nebo grafické vizualizace, tento výstup se zobrazí pod buňkou. Funkce clear_output umožňuje tento výstup vymazat.

Implementace funkce není v úryvku kódu uvedena, jak naznačuje trojtečka (...). Trojtečka představuje zástupný symbol pro skutečný kód, který provádí vymazání výstupu. Implementace funkce může zahrnovat interakci s API Jupyter Notebooku nebo IPythonu za účelem odstranění stávajícího výstupu z buňky.

Celkově tato funkce poskytuje pohodlný způsob, jak vymazat výstup aktuální buňky v Jupyter Notebooku nebo IPythonu, což usnadňuje správu a aktualizaci zobrazeného výstupu během interaktivních programovacích sezení.


In [ ]:
# Download Phi-3-mini-4k-instruct model & Whisper Tiny
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

#whisper for speech to text()
import whisper
select_model ="tiny" # ['tiny', 'base']
whisper_model = whisper.load_model(select_model)

#from IPython.display import clear_output
#clear_output()

Proveďte převod textu na řeč (TTS) pomocí služby Edge TTS. Pojďme si projít relevantní implementace funkcí jednu po druhé:

1. `calculate_rate_string(input_value)`: Tato funkce přijímá vstupní hodnotu a vypočítává řetězec rychlosti pro hlas TTS. Vstupní hodnota představuje požadovanou rychlost řeči, kde hodnota 1 odpovídá normální rychlosti. Funkce vypočítává řetězec rychlosti odečtením 1 od vstupní hodnoty, násobením výsledku číslem 100 a následným určením znaménka na základě toho, zda je vstupní hodnota větší nebo rovna 1. Funkce vrací řetězec rychlosti ve formátu "{sign}{rate}".

2. `make_chunks(input_text, language)`: Tato funkce přijímá vstupní text a jazyk jako parametry. Rozděluje vstupní text na části na základě pravidel specifických pro daný jazyk. V této implementaci, pokud je jazyk "English", funkce rozdělí text na každé tečce ("."), odstraní jakékoli počáteční nebo koncové mezery, přidá tečku ke každé části a vrátí filtrovaný seznam částí.

3. `tts_file_name(text)`: Tato funkce generuje název souboru pro audio soubor TTS na základě vstupního textu. Provádí několik transformací na text: odstraní koncovou tečku (pokud je přítomna), převede text na malá písmena, odstraní počáteční a koncové mezery a nahradí mezery podtržítky. Poté text zkrátí na maximálně 25 znaků (pokud je delší) nebo použije celý text, pokud je prázdný. Nakonec vygeneruje náhodný řetězec pomocí modulu [`uuid`] a spojí jej se zkráceným textem, aby vytvořil název souboru ve formátu "/content/edge_tts_voice/{truncated_text}_{random_string}.mp3".

4. `merge_audio_files(audio_paths, output_path)`: Tato funkce sloučí více audio souborů do jednoho. Přijímá seznam cest k audio souborům a výstupní cestu jako parametry. Funkce inicializuje prázdný objekt `AudioSegment` nazvaný [`merged_audio`]. Poté iteruje přes každou cestu k audio souboru, načte audio soubor pomocí metody `AudioSegment.from_file()` z knihovny `pydub` a připojí aktuální audio soubor k objektu [`merged_audio`]. Nakonec exportuje sloučené audio do zadané výstupní cesty ve formátu MP3.

5. `edge_free_tts(chunks_list, speed, voice_name, save_path)`: Tato funkce provádí operaci TTS pomocí služby Edge TTS. Přijímá seznam textových částí, rychlost řeči, název hlasu a cestu pro uložení jako parametry. Pokud je počet částí větší než 1, funkce vytvoří adresář pro uložení jednotlivých audio souborů částí. Poté iteruje přes každou část, sestaví příkaz Edge TTS pomocí funkce `calculate_rate_string()`, názvu hlasu a textu části a provede příkaz pomocí funkce `os.system()`. Pokud je provedení příkazu úspěšné, přidá cestu k vygenerovanému audio souboru do seznamu. Po zpracování všech částí sloučí jednotlivé audio soubory pomocí funkce `merge_audio_files()` a uloží sloučené audio do zadané cesty. Pokud je pouze jedna část, přímo vygeneruje příkaz Edge TTS a uloží audio do zadané cesty. Nakonec vrátí cestu k vygenerovanému audio souboru.

6. `random_audio_name_generate()`: Tato funkce generuje náhodný název audio souboru pomocí modulu [`uuid`]. Vygeneruje náhodné UUID, převede jej na řetězec, vezme prvních 8 znaků, připojí příponu ".mp3" a vrátí náhodný název audio souboru.

7. `talk(input_text)`: Tato funkce je hlavním vstupním bodem pro provedení operace TTS. Přijímá vstupní text jako parametr. Nejprve zkontroluje délku vstupního textu, aby určila, zda se jedná o dlouhou větu (větší nebo rovnou 600 znaků). Na základě délky a hodnoty proměnné `translate_text_flag` určí jazyk a vygeneruje seznam textových částí pomocí funkce `make_chunks()`. Poté vygeneruje cestu pro uložení audio souboru pomocí funkce `random_audio_name_generate()`. Nakonec zavolá funkci `edge_free_tts()` pro provedení operace TTS a vrátí cestu k vygenerovanému audio souboru.

Celkově tyto funkce spolupracují na rozdělení vstupního textu na části, generování názvu souboru pro audio soubor, provedení operace TTS pomocí služby Edge TTS a sloučení jednotlivých audio souborů do jednoho.


In [ ]:
#@title Edge TTS
def calculate_rate_string(input_value):
    rate = (input_value - 1) * 100
    sign = '+' if input_value >= 1 else '-'
    return f"{sign}{abs(int(rate))}"


def make_chunks(input_text, language):
    language="English"
    if language == "English":
      temp_list = input_text.strip().split(".")
      filtered_list = [element.strip() + '.' for element in temp_list[:-1] if element.strip() and element.strip() != "'" and element.strip() != '"']
      if temp_list[-1].strip():
          filtered_list.append(temp_list[-1].strip())
      return filtered_list


import re
import uuid
def tts_file_name(text):
    if text.endswith("."):
        text = text[:-1]
    text = text.lower()
    text = text.strip()
    text = text.replace(" ","_")
    truncated_text = text[:25] if len(text) > 25 else text if len(text) > 0 else "empty"
    random_string = uuid.uuid4().hex[:8].upper()
    file_name = f"/content/edge_tts_voice/{truncated_text}_{random_string}.mp3"
    return file_name


from pydub import AudioSegment
import shutil
import os
def merge_audio_files(audio_paths, output_path):
    # Initialize an empty AudioSegment
    merged_audio = AudioSegment.silent(duration=0)

    # Iterate through each audio file path
    for audio_path in audio_paths:
        # Load the audio file using Pydub
        audio = AudioSegment.from_file(audio_path)

        # Append the current audio file to the merged_audio
        merged_audio += audio

    # Export the merged audio to the specified output path
    merged_audio.export(output_path, format="mp3")

def edge_free_tts(chunks_list,speed,voice_name,save_path):
  # print(chunks_list)
  if len(chunks_list)>1:
    chunk_audio_list=[]
    if os.path.exists("/content/edge_tts_voice"):
      shutil.rmtree("/content/edge_tts_voice")
    os.mkdir("/content/edge_tts_voice")
    k=1
    for i in chunks_list:
      print(i)
      edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{i}" --write-media /content/edge_tts_voice/{k}.mp3'
      print(edge_command)
      var1=os.system(edge_command)
      if var1==0:
        pass
      else:
        print(f"Failed: {i}")
      chunk_audio_list.append(f"/content/edge_tts_voice/{k}.mp3")
      k+=1
    # print(chunk_audio_list)
    merge_audio_files(chunk_audio_list, save_path)
  else:
    edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{chunks_list[0]}" --write-media {save_path}'
    print(edge_command)
    var2=os.system(edge_command)
    if var2==0:
      pass
    else:
      print(f"Failed: {chunks_list[0]}")
  return save_path

# text = "This is Microsoft Phi 3 mini 4k instruct Demo" Simply update the text variable with the text you want to convert to speech
text = 'This is Microsoft Phi 3 mini 4k instruct Demo'  # @param {type: "string"}
Language = "English" # @param ['English']
# Gender of voice simply change from male to female and choose the voice you want to use
Gender = "Female"# @param ['Male', 'Female']
female_voice="en-US-AriaNeural"# @param["en-US-AriaNeural",'zh-CN-XiaoxiaoNeural','zh-CN-XiaoyiNeural']
speed = 1  # @param {type: "number"}
translate_text_flag  = False
if len(text)>=600:
  long_sentence = True
else:
  long_sentence = False

# long_sentence = False # @param {type:"boolean"}
save_path = ''  # @param {type: "string"}
if len(save_path)==0:
  save_path=tts_file_name(text)
if Language == "English" :
  if Gender=="Male":
    voice_name="en-US-ChristopherNeural"
  if Gender=="Female":
    voice_name=female_voice
    # voice_name="en-US-AriaNeural"


if translate_text_flag:
  input_text=text
  # input_text=translate_text(text, Language)
  # print("Translateting")
else:
  input_text=text
if long_sentence==True and translate_text_flag==True:
  chunks_list=make_chunks(input_text,Language)
elif long_sentence==True and translate_text_flag==False:
  chunks_list=make_chunks(input_text,"English")
else:
  chunks_list=[input_text]
# print(chunks_list)
# edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
# from IPython.display import clear_output
# clear_output()
# from IPython.display import Audio
# Audio(edge_save_path, autoplay=True)

from IPython.display import clear_output
from IPython.display import Audio
if not os.path.exists("/content/audio"):
    os.mkdir("/content/audio")
import uuid
def random_audio_name_generate():
  random_uuid = uuid.uuid4()
  audio_extension = ".mp3"
  random_audio_name = str(random_uuid)[:8] + audio_extension
  return random_audio_name
def talk(input_text):
  global translate_text_flag,Language,speed,voice_name
  if len(input_text)>=600:
    long_sentence = True
  else:
    long_sentence = False

  if long_sentence==True and translate_text_flag==True:
    chunks_list=make_chunks(input_text,Language)
  elif long_sentence==True and translate_text_flag==False:
    chunks_list=make_chunks(input_text,"English")
  else:
    chunks_list=[input_text]
  save_path="/content/audio/"+random_audio_name_generate()
  edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
  return edge_save_path


edge_save_path=talk(text)
Audio(edge_save_path, autoplay=True)

Implementace dvou funkcí: convert_to_text a run_text_prompt, stejně jako deklarace dvou tříd: str a Audio.

Funkce convert_to_text přijímá jako vstup audio_path a přepisuje zvuk na text pomocí modelu nazvaného whisper_model. Funkce nejprve ověřuje, zda je příznak gpu nastaven na True. Pokud ano, whisper_model se použije s určitými parametry, jako jsou word_timestamps=True, fp16=True, language='English' a task='translate'. Pokud je příznak gpu nastaven na False, whisper_model se použije s fp16=False. Výsledný přepis je poté uložen do souboru s názvem 'scan.txt' a vrácen jako text.

Funkce run_text_prompt přijímá jako vstup message a chat_history. Používá funkci phi_demo k vygenerování odpovědi od chatbotu na základě vstupní zprávy. Vygenerovaná odpověď je poté předána funkci talk, která odpověď převede na zvukový soubor a vrátí cestu k souboru. Třída Audio se používá k zobrazení a přehrání zvukového souboru. Zvuk je zobrazen pomocí funkce display z modulu IPython.display a objekt Audio je vytvořen s parametrem autoplay=True, takže zvuk začne automaticky přehrávat. chat_history je aktualizován o vstupní zprávu a vygenerovanou odpověď, a vrácen je prázdný řetězec a aktualizovaný chat_history.

Třída str je vestavěná třída v Pythonu, která reprezentuje sekvenci znaků. Poskytuje různé metody pro manipulaci a práci s řetězci, jako jsou capitalize, casefold, center, count, encode, endswith, expandtabs, find, format, index, isalnum, isalpha, isascii, isdecimal, isdigit, isidentifier, islower, isnumeric, isprintable, isspace, istitle, isupper, join, ljust, lower, lstrip, partition, replace, removeprefix, removesuffix, rfind, rindex, rjust, rpartition, rsplit, rstrip, split, splitlines, startswith, strip, swapcase, title, translate, upper, zfill a další. Tyto metody umožňují provádět operace jako vyhledávání, nahrazování, formátování a manipulaci s řetězci.

Třída Audio je vlastní třída, která reprezentuje zvukový objekt. Používá se k vytvoření přehrávače zvuku v prostředí Jupyter Notebook. Třída přijímá různé parametry, jako jsou data, filename, url, embed, rate, autoplay a normalize. Parametr data může být numpy pole, seznam vzorků, řetězec reprezentující název souboru nebo URL, nebo surová PCM data. Parametr filename se používá k určení místního souboru, ze kterého se načtou zvuková data, a parametr url se používá k určení URL, ze kterého se stáhnou zvuková data. Parametr embed určuje, zda by zvuková data měla být vložena pomocí data URI nebo odkazována z původního zdroje. Parametr rate specifikuje vzorkovací frekvenci zvukových dat. Parametr autoplay určuje, zda by se zvuk měl začít automaticky přehrávat. Parametr normalize specifikuje, zda by zvuková data měla být normalizována (přepočítána) na maximální možný rozsah. Třída Audio také poskytuje metody jako reload pro opětovné načtení zvukových dat ze souboru nebo URL, a atributy jako src_attr, autoplay_attr a element_id_attr pro získání odpovídajících atributů pro zvukový prvek v HTML.

Celkově jsou tyto funkce a třídy používány k přepisu zvuku na text, generování zvukových odpovědí od chatbotu a zobrazení a přehrání zvuku v prostředí Jupyter Notebook.


In [ ]:
#@title Run gradio app
def convert_to_text(audio_path):
  gpu=True
  if gpu:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=True,language='English',task='translate')
  else:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=False,language='English',task='translate')
  with open('scan.txt', 'w') as file:
    file.write(str(result))
  return result["text"]


import gradio as gr
from IPython.display import Audio, display
def run_text_prompt(message, chat_history):
    bot_message = phi_demo(message)
    edge_save_path=talk(bot_message)
    # print(edge_save_path)
    display(Audio(edge_save_path, autoplay=True))

    chat_history.append((message, bot_message))
    return "", chat_history


def run_audio_prompt(audio, chat_history):
    if audio is None:
        return None, chat_history
    print(audio)
    message_transcription = convert_to_text(audio)
    _, chat_history = run_text_prompt(message_transcription, chat_history)
    return None, chat_history


with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Chat with Phi 3 mini 4k instruct")

    msg = gr.Textbox(label="Ask anything")
    msg.submit(run_text_prompt, [msg, chatbot], [msg, chatbot])

    with gr.Row():
        audio = gr.Audio(sources="microphone", type="filepath")

        send_audio_button = gr.Button("Send Audio", interactive=True)
        send_audio_button.click(run_audio_prompt, [audio, chatbot], [audio, chatbot])

demo.launch(share=True,debug=True)


---

**Prohlášení**:  
Tento dokument byl přeložen pomocí služby pro automatický překlad [Co-op Translator](https://github.com/Azure/co-op-translator). I když se snažíme o přesnost, mějte prosím na paměti, že automatické překlady mohou obsahovat chyby nebo nepřesnosti. Původní dokument v jeho původním jazyce by měl být považován za autoritativní zdroj. Pro důležité informace se doporučuje profesionální lidský překlad. Neodpovídáme za žádná nedorozumění nebo nesprávné interpretace vyplývající z použití tohoto překladu.
